# Geospatial BI & Data Viz Companies - Part 2

- VISUALIZATION PROJECT Geospatial Business Intelligence (BI)
    * Make a geospatial analysis of the `companies` dataset
    * Things you know:
        - You have a software company with 50 employees
        - The company creates video games
        - Roles in your company: 20 developers, 20 Designers/Creatives/UX/UI and 10 executives/managers
    * Do an analysis about placing the new company offices in the best environment based on the following criteria:
        - There should be software engineers working around
        - The surroundings must have a good ratio of big companies vs startups
        - Ensure you have in your surroundings companies that cover the interests of your team
        - Avoid old companies, prefer recently created ones
        
JSON companies --> MongoDB --> Python:
Queries en Jupyter-Notebook --> DataFrame (Pandas) --> columna geopoint con diccionario con type = point y un array con formato array [lat,long]
indice geospatial (point) para volver a cargar en MongoDB y hacer nuevas queries (esta vez tocar la parte de queries geo-espaciales $near, etc...) 
por ej: de todos los puntos, dime el que está más cerca de empresas de videojuegos, o a 1000m
para tener el max y min, hay que probarlos todos 

In [1]:
# import pymongo to connect Python with MongoDB
from pymongo import MongoClient
# to work with stats
import pandas as pd
# to work with dataframes
import numpy as np
# to work with json
import json
from pandas.io.json import json_normalize
from flatten_json import flatten
# to plot
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

### 1. Prepare the data: extract from the companies database the relevant information for the challenge.

* company identification
    - id (just in case we would need to get more information later)
    - name
* only currently active company
    - 'deadpooled_year' --> not none
    
* for geolocation analysis & geospatial visualization
    - offices.latitude, offices.longitude --> not null
    - offices.country_code, offices.city --> not null

* to determine if the company is old (negative) o recent (positive)
    - founded_year --> not null

* to define if it's a small (startup) or a big company:
    - number_of_employees --> company size (make sure there are not ficticious and have >= 1 employee)
    - investments.funding_round.round_code: 'Angel','seed' --> to associate to startup category
    https://support.crunchbase.com/hc/en-us/articles/115010458467-Glossary-of-Funding-Types
    - investments.funding_round.funded_year
    - ipo.pub_year,ipo.valuation_amount --> to associate with big company   

* to match with our team interests: technology & videogames
    - category_code: 'software', 'web', 'games_video' --> to filter as 'best match' for our team --> not null
    - description: 'software','technology', 'Platform','Social network' --> for a qualitative analysis
    - tag_list: 'network', 'online-communities','projects', etc --> for a qualitative analysis

In [ ]:
# connecting on default host and port
client = MongoClient ('localhost', 27017)

# loading the database
db = client['companies']

# getting the collection
companies = db['companies']

# defining the main initial query

query1 = db.companies.find({'$and':[{'founded_year':{'$ne': None}},
                                   {'category_code':{'$ne': None}},
                                   {'deadpooled_year':None},
                                   {'number_of_employees':{'$gte':1}},
                                   {'offices.latitude':{'$ne':  None}},
                                   {'offices.longitude': {'$ne':  None}},
                                   {'offices.country_code':{'$ne':  None}},
                                   {'offices.city':{'$ne':  None}}]},
{'_id':1, 'name':1,'founded_year':1,'deadpooled_year':1,'number_of_employees':1,
 'category_code':1,'description':1,'tag_list':1
 'offices.latitude':1,'offices.longitude':1,'offices.country_code':1,'offices.city':1,
'investments.funding_round.round_code':1,'investments.funding_round.funded_year':1,
 'ipo.pub_year':1,'ipo.valuation_amount':1})

In [ ]:
# 1st query to get the offices information per company and normalize JSON
query1 = db.companies.find({'$and':[{'founded_year':{'$ne': None}},{'category_code':{'$ne': None}},
{'deadpooled_year':None},{'number_of_employees':{'$gte':1}},{'offices.latitude':{'$ne':  None}},{'offices.longitude': {'$ne':  None}},
{'offices.country_code':{'$ne':  None}},{'offices.city':{'$ne':  None}}]},{'_id':1, 'name':1,'founded_year':1,'deadpooled_year':1, 
'number_of_employees':1,'offices.latitude':1,'offices.longitude':1,'offices.country_code':1,'offices.city':1,
'category_code':1,'description':1,'tag_list':1})


In [ ]:
offices = json_normalize(data=query1, 
                         record_path='offices', 
                         meta=['_id','name', 'founded_year','number_of_employees', 'category_code', 'deadpooled_year', 'description', 'tag_list'], 
                         errors='ignore')

offices.head(20)

In [ ]:
# 2nd query to get the startups funding information per company and normalize JSON
query2 = db.companies.find({'$and':[{'founded_year':{'$ne': None}},{'category_code':{'$ne': None}},
{'deadpooled_year':None},{'number_of_employees':{'$gte':1}},{'offices.latitude':{'$ne':  None}},{'offices.longitude': {'$ne':  None}},
{'offices.country_code':{'$ne':  None}},{'offices.city':{'$ne':  None}}]},{'_id':1, 'name':1,'founded_year':1,'deadpooled_year':1, 
'number_of_employees':1,'investments.funding_round.round_code':1,'investments.funding_round.funded_year':1,
'category_code':1,'description':1,'tag_list':1})
# pd.DataFrame(query2).iloc[4]['investments']
list(query2)[0].keys()
# $exists : True

In [ ]:
query2 
fundings = json_normalize(pd.concat([pd.DataFrame(i) for i in query2['investments']]).to_dict('investments'))

In [ ]:
fundings = json_normalize(data=query2)
fundings.iloc[4]

In [ ]:
# 3rd query to get the IPO valuation information per company and normalize JSON
query3 = db.companies.find({'$and':[{'founded_year':{'$ne': None}},{'category_code':{'$ne': None}},
{'deadpooled_year':None},{'number_of_employees':{'$gte':1}},{'offices.latitude':{'$ne':  None}},{'offices.longitude': {'$ne':  None}},
{'offices.country_code':{'$ne':  None}},{'offices.city':{'$ne':  None}},{'ipo':{'$ne':  None}}]},{'_id':1, 'name':1,'founded_year':1,'deadpooled_year':1, 
'number_of_employees':1,'ipo':1,'category_code':1,'description':1,'tag_list':1})

valuations = pd.DataFrame(query3)
valuations.head(20)

In [ ]:
# we load our query to a dataframe to work with                           
def cursor_to_df(query):
    return pd.DataFrame(list(query))

raw_data = cursor_to_df(query)

In [ ]:
raw_data2.head()

In [ ]:
# checking we get all the requested info
raw_data.columns

In [ ]:
# checking what kind of variables we have
raw_data.dtypes

In [ ]:
raw_data.head()

In [ ]:
# check all variables we ask without null have 100% data per register + all companies are currently active
raw_data.info()

In [ ]:
def normalize(raw df,norm_df):
    locations = json_normalize(bks["address"])
    gelocs = locations.apply(lambda e: [e["coord"][0],e["coord"][1]], result_type="expand", axis=1)
    clean_df = pd.concat([bks["restaurant_id"],locations[["street","zipcode"]],gelocs], axis=1)
    clean_df.rename({1:"lat",0:"long"}, axis=1, inplace=True)
    return clean_df

In [ ]:
data = json_normalize(data= raw_data, record_path='offices', 
                      meta=['_id', 'category_code', 'deadpooled_year', 'description','founded_year', 'investments', 'ipo', 'name', 'number_of_employees','tag_list'])

### Working with Founded year & Nb employees: convert numerical variables into bins as decision parameters

In [ ]:
cutoffs_tab = data.describe()
cutoffs_tab

In [ ]:
# looking for outliers first:
sns.boxplot(x=data['number_of_employees'])
# in this case we want a great ratio between big companies and start ups, it's preferible not to remove the big 4

In [ ]:
sns.boxplot(x=data['founded_year'])
# in this case we want to avoid old companies so it's recommended not to remove them from dataset

In [ ]:
data_justforplot = data[(data['founded_year']>= data.founded_year.quantile(0.25)) & (data['number_of_employees']<=data.number_of_employees.quantile(0.75))]

In [ ]:
data_justforplot.number_of_employees.hist(bins=20)
plt.suptitle('Zoom on 75% companies - nb of employees')

In [ ]:
# First we have a look on these variables
data_justforplot.founded_year.hist(bins=20)
plt.suptitle('Zoom on 75% companies - founded year')

In [ ]:
# creating a function to apply bins with Q1,Q2,Q3 and Q4 and be able to automate when refreshing database
def bins_q4(df,var):
    bins_labels = ['q1','q2','q3','q4']
    cutoffs = [cutoffs_tab[var]['min'],cutoffs_tab[var]['25%'],cutoffs_tab[var]['50%'],cutoffs_tab[var]['75%'],cutoffs_tab[var]['max']]
    return pd.cut(df[var],cutoffs, labels=bins_labels)

In [ ]:
# we apply the bins to each var
data['founded_year_bins'] = bins_q4(data,'founded_year')
# and we check
data['founded_year_bins'].value_counts()

In [ ]:
# same for this one
data['number_of_employees_bins'] = bins_q4(data,'number_of_employees')
data['number_of_employees_bins'].value_counts()

### Normalizing Offices JSON

In [ ]:
raw_data = raw_data['offices'][raw_data.columns]
dict_flattened = (flatten(record, '.') for record in data)
data = pd.DataFrame(dict_flattened)
print(data)

In [ ]:
data_norm = json_normalize (data=query, record_path=['offices'], meta=['name', 'category_code','number_of_employees','founded_year'], errors='ignore')

In [ ]:
def flatten_data(raw_data):
    dic_flattened = [flatten(d) for d in raw_data]
    data = pd.DataFrame(dic_flattened)
    return data

In [ ]:
raw_data['offices'] = flatten_data(raw_data['offices'])
raw_data['offices'].head()

In [ ]:
def flatten_json(col):
    out = {}

    def flatten(row, name=''):
        if type(row) is dict:
            for x in row:
                flatten(row[x], name + x + '_')
        elif type(row) is list:
            i = 0
            for x in row:
                flatten(x, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = row

    flatten(col)
    return out

In [ ]:
data = raw_data['offices'].append(listOfSeries , ignore_index=True)

In [ ]:
flat = flatten_json(sample_object2)
json_normalize(flat)

In [ ]:
df = json_normalize (data=raw_data, record_path=['offices'], meta=['name', 'category_code','number_of_employees','founded_year'], errors='ignore')

In [ ]:
def office_per_company(offices):
    company_df = pd.DataFrame.from_records(raw_data['offices'])
    for company in raw_data['name']:
        company_df = raw_data['offices'].apply(pd.DataFrame.from_records)
raw_data['offices']

In [ ]:
data = json_normalize(raw_data, record_path='offices')

In [ ]:
data = json_normalize(raw_data, record_path='offices', meta=['_id','category_code', 'deadpooled_year', 'description','founded_year', 'investments', 'ipo', 'name', 'number_of_employees', 'tag_list'])

In [ ]:
# Defining 'Startups': Less than 10 employees

In [ ]:
# checking the data quantity for our analyse
data.shape

In [ ]:
data.category_code.value_counts()

In [ ]:
data.to_json(‘./companies.json’, orient=‘records’, lines=True)